In [89]:
import numpy as np

We note that we are using the base field `QQ` for all computations currently

In [90]:
# Let us initialize constants and vector spaces.

N = 16
n = 2
d = 4
indeterminates = [var("".join(["x_", str(i), str(j), str(k), str(l)])) for i in range(n) for j in range(n) for k in range(n) for l in range(n)]
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

glN = MatrixSpace(QQ, N)
dimV = binomial(N + d - 1, d)
V = VectorSpace(QQ, dimV)

Defining x_0000, x_0001, x_0010, x_0011, x_0100, x_0101, x_0110, x_0111, x_1000, x_1001, x_1010, x_1011, x_1100, x_1101, x_1110, x_1111


In [91]:
# This cell defines the explicit isomorphism between V <=> R^d

# construct monomial list
i = 0
elem = 0
lst = []
while i < N*d:
    lst.append(indeterminates[elem])
    i += 1
    if i % d == 0:
        elem += 1
S = Subsets(lst, d, submultiset=True).list()
monomials = [R(np.prod(S[i])) for i in range(dimV)]

# R^d => V
def polynomial_to_vector(p):
    if not p in R:
        return 0
    coefficient_list = []
    for i in range(len(monomials)):
        coefficient_list.append(p.monomial_coefficient(monomials[i]))
    return V(coefficient_list)

# V => R^d
def vector_to_polynomial(v):
    if not v in V:
        return 0
    p = 0
    for i, item in enumerate(Sequence(monomials)):
        p += v[i]*item
    return R(p)

In [92]:
# Data processing cell to flatten sparse vectors over V. We also create an optimized inner product function on such vectors.

# V => V
def sparsify(v):
    flattened_list = []
    for index, value in enumerate(v):
        if v[index] != 0:
            flattened_list.append((index,value))
    return flattened_list


# <V,V> => QQ
def sparse_inner_product(a,b):
    i = 0; j = 0
    result = 0
    while i < len(a) and j < len(b):
        if a[i][0] == b[j][0]:
            result += a[i][1]*b[j][1]
            i += 1
            j += 1
        elif a[i][0] < b[j][0]:
            i += 1
        else:
            j += 1
    return result

In [93]:
# construct f

M = MatrixSpace(R, n^2)

x_sub = lambda i,j,k,l: R(var("".join(["x_", str(i), str(j), str(k), str(l)])))

A_12 = M(0)
A_23 = M(0)
A_13 = M(0)
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                A_12[2*i + j, 2*k + l] = x_sub(i,j,k,l)
                A_23[2*j + k, 2*i + l] = x_sub(i,j,k,l)
                A_13[2*i + k, 2*j + l] = x_sub(i,j,k,l)

f = [R(A_12.determinant()),R(A_23.determinant()),R(A_13.determinant())]
grad_f = [fi.gradient() for fi in f]

In [94]:
f_vec = [polynomial_to_vector(fi) for fi in f]
f_vec_sliced = f_vec[:2]

# CHECK: three determinant polynomials are linearly dependent
print(len(V.linear_dependence(f_vec)) > 0)
print(len(V.linear_dependence(f_vec_sliced)) > 0)

True
False


In [95]:
# This defines the action of glN (general linear lie algebra) on the homogeneous polynomials of degree d in R.

# X_action: (glN, R^d) => R^d
def X_action(X, i):
    sum = 0
    for a in range(N):
        for b in range(N):
            sum += -X[a][b]*indeterminates[b]*grad_f[i][a]
    return R(sum)

In [96]:
# define W = span{f}, W_perp = orthogonal complement of span{f}

W = V.subspace(f_vec_sliced)
W_perp = W.complement()

print(W)
print(W_perp)

g = W_perp.basis()
p = W.dimension()
q = W_perp.dimension()

Vector space of degree 3876 and dimension 2 over Rational Field
Basis matrix:
2 x 3876 dense matrix over Rational Field
Vector space of degree 3876 and dimension 3874 over Rational Field
Basis matrix:
3874 x 3876 dense matrix over Rational Field


We note `M_f` $:= \tilde{M}_f$

In [97]:
# Pre-processing memoization of entries of M_f

# returns E_ij basis matrix of glN
E = lambda i,j: list(glN.basis())[(N*i) + j]

# E_action_f[l][i][j] = (E_ij)(f_l)
E_action_f = [[[sparsify(polynomial_to_vector(X_action(E(i,j),l))) for i in range(N)] for j in range(N)] for l in range(p)]

# convert g to sparse representation
g_sparse = [sparsify(g[i]) for i in range(q)]

In [64]:

# Construct M_f
M_f = zero_matrix(QQ, p*q, N^2)
for l in range(p):
    for k in range(q):
        for i in range(N):
            for j in range(N):
                M_f[(p*l) + k,(N*i) + j] = sparse_inner_product(E_action_f[l][i][j], g_sparse[k])

In [72]:
print('rank(M_f) =', M_f.rank())
print('dim(ker(M_f)) =', glN.dimension() - M_f.rank())

rank(M_f) = 225
dim(ker(M_f)) = 31
